In [1]:
import pandas as pd
from os import listdir
from os.path import isfile, join
import numpy as np

In [35]:
# get data in desirved format
dow_files_addr = "./Data/Dow_30_1_min/"
etf_files_addr = "./Data/50_ETFs_1min/"

dow_files_names = [f for f in listdir(dow_files_addr)]
etf_files_names = [f for f in listdir(etf_files_addr)]

dow_names = [f.split(".")[0] for f in dow_files_names]
etf_names = [f.split(".")[0] for f in etf_files_names]

dow_dfs = []
etf_dfs = []
headers = ["date", "time", "open_price", "close_price", "highest_price", "lowest_price", "volumn"]
dtypes={"date": "str", 
    "time":"str",
    "open_price": "float", 
    "highest_price": "float",
    "lowest_price": "float", 
    "close_price": "float",
    "volumn": "int"}
for dow_name in dow_files_names:
    df = pd.read_csv(dow_files_addr + dow_name, sep=",",names=headers,dtype=dtypes, header=None)
    df["datetime"] = df["date"] + " " + df["time"]
    df["datetime"] = pd.to_datetime(df["datetime"], format="%m/%d/%Y %H:%M")
    dow_dfs.append(df)
for etf_name in etf_files_names:
    df = pd.read_csv(etf_files_addr + etf_name, sep=",",names=headers,dtype=dtypes, header=None)
    df["datetime"] = df["date"] + " " + df["time"]
    df["datetime"] = pd.to_datetime(df["datetime"], format="%m/%d/%Y %H:%M")
    etf_dfs.append(df)

In [ ]:
# get start and end for both dow and etf
dow_time_slice = [pd.datetime.min, pd.datetime.max]
etf_time_slice = [pd.datetime.min, pd.datetime.max]
for df in dow_dfs:
    start = min(df["datetime"])
    if start < dow_time_slice[0]:
        dow_time_slice[0] = start
    end = max(df["datetime"])
    if end > dow_time_slice[1]:
        dow_time_slice = end
for df in etf_dfs:
    start = min(df["datetime"])
    if start < etf_time_slice[0]:
        etf_time_slice[0] = start
    end = max(df["datetime"])
    if end > etf_time_slice[1]:
        etf_time_slice = end

    

/usr/lib/python3/dist-packages/ipykernel_launcher.py:2: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime instead.
  
/usr/lib/python3/dist-packages/ipykernel_launcher.py:3: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime instead.
  This is separate from the ipykernel package so we can avoid doing imports until


In [44]:
pd.datetime.min

/usr/lib/python3/dist-packages/ipykernel_launcher.py:1: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime instead.
  """Entry point for launching an IPython kernel.


datetime.datetime(1, 1, 1, 0, 0)

In [43]:
pd.datetime.max

/usr/lib/python3/dist-packages/ipykernel_launcher.py:1: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime instead.
  """Entry point for launching an IPython kernel.


datetime.datetime(9999, 12, 31, 23, 59, 59, 999999)

In [36]:
tmp = dow_dfs[0]

In [37]:
tmp

,date,time,open_price,close_price,highest_price,lowest_price,volumn,datetime
0,04/27/2007,10:52,73.72,73.74,73.72,73.74,7405,2007-04-27 10:52:00
1,04/27/2007,10:53,73.74,73.80,73.74,73.80,2323,2007-04-27 10:53:00
2,04/27/2007,10:54,73.78,73.80,73.78,73.78,3340,2007-04-27 10:54:00
3,04/27/2007,10:55,73.78,73.80,73.78,73.80,2904,2007-04-27 10:55:00
4,04/27/2007,10:56,73.80,73.82,73.78,73.82,5324,2007-04-27 10:56:00
...,...,...,...,...,...,...,...,...
1470596,06/05/2020,19:44,12.05,12.05,12.05,12.05,384,2020-06-05 19:44:00
1470597,06/05/2020,19:52,12.05,12.05,12.05,12.05,416,2020-06-05 19:52:00
1470598,06/05/2020,19:55,12.04,12.04,12.04,12.04,1791,2020-06-05 19:55:00
1470599,06/05/2020,19:56,12.00,12.00,12.00,12.00,400,2020-06-05 19:56:00


In [38]:
min(tmp["datetime"])

Timestamp('2007-04-27 10:52:00')

In [39]:
max(tmp["datetime"])

Timestamp('2020-06-05 19:59:00')